<a href="https://colab.research.google.com/github/abdullahzunorain/Making_an_end-to-end-project_on_colab_using_streamlit/blob/main/Weather_Based_Outfit_Suggestion_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install groq requests gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggin

In [19]:
from google.colab import userdata
GROQ_API_TOKEN = userdata.get('GROQ_API_TOKEN')


In [20]:
from google.colab import userdata
WeatherApp = userdata.get('WeatherApp')


In [4]:
from google.colab import userdata

# Retrieve the GROQ API Token and WeatherApp API Key
GROQ_API_TOKEN = userdata.get('GROQ_API_TOKEN')
WeatherApp = userdata.get('WeatherApp')

# Confirm that the keys are accessed (print statements should be avoided for sensitive data in production)
print("GROQ API Token:", "Loaded Successfully" if GROQ_API_TOKEN else "Not Found")
print("WeatherApp Key:", "Loaded Successfully" if WeatherApp else "Not Found")


GROQ API Token: Loaded Successfully
WeatherApp Key: Loaded Successfully


In [5]:
import requests

def get_weather_data(location, api_key):
    # OpenWeatherMap API endpoint
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"

    response = requests.get(url)
    data = response.json()

    if response.status_code == 200:
        return {
            "location": data["name"],
            "temperature": data["main"]["temp"],
            "weather": data["weather"][0]["description"],
            "humidity": data["main"]["humidity"],
            "wind_speed": data["wind"]["speed"]
        }
    else:
        return {"error": data.get("message", "An error occurred")}


In [6]:
import os
from groq import Groq

# Setup API key
client = Groq(api_key=GROQ_API_TOKEN) #os.environ.get("GROQ_API_KEY"))

def generate_outfit_suggestion(weather_data):
    prompt = f"Suggest an outfit for a person based on the following weather data: {weather_data}"

    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-8b-8192"
    )

    return chat_completion.choices[0].message.content


In [7]:
import gradio as gr

def suggest_outfit(location, weather_api_key):
    # Get weather data
    weather_data = get_weather_data(location, weather_api_key)

    if "error" in weather_data:
        return weather_data["error"]

    # Generate outfit suggestion using LLAMA
    outfit_suggestion = generate_outfit_suggestion(weather_data)
    return outfit_suggestion

interface = gr.Interface(
    fn=suggest_outfit,
    inputs=[
        gr.Textbox(label="Enter Your Location"),
        gr.Textbox(label="OpenWeatherMap API Key (Optional)", type="password")
    ],
    outputs="text",
    live=True
)

interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6fe7d85130e8567fb1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


___

application

In [8]:
# Install necessary libraries
!pip install groq requests streamlit gradio

# Import Libraries and Set API Keys
import os
import requests
from groq import Groq
import gradio as gr
from google.colab import userdata

# Retrieve the secret keys securely
GROQ_API_KEY = userdata.get('GROQ_API_TOKEN')
WEATHER_API_KEY = userdata.get('WeatherApp')

# Set environment variables for the APIs
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

# Fetch Weather Data from OpenWeatherMap
def get_weather_data(location, api_key):
    """Fetch weather data from OpenWeatherMap API"""
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
    response = requests.get(url)
    data = response.json()

    if response.status_code == 200:
        return {
            "location": data["name"],
            "temperature": data["main"]["temp"],
            "weather": data["weather"][0]["description"],
            "humidity": data["main"]["humidity"],
            "wind_speed": data["wind"]["speed"]
        }
    else:
        return {"error": data.get("message", "An error occurred")}

# Generate Outfit Suggestions using Groq LLAMA model
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def generate_outfit_suggestion(weather_data):
    """Generate outfit suggestion based on weather data using LLAMA model"""
    prompt = f"Suggest an outfit for a person based on the following weather data: {weather_data}"

    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-8b-8192"  # You can use a smaller model if needed, such as llama3-1b
    )

    return chat_completion.choices[0].message.content

# Suggest Outfit Function combining both weather and LLAMA model
def suggest_outfit(location, weather_api_key):
    """Function to combine weather data and Groq model for outfit suggestion"""
    # Get weather data
    weather_data = get_weather_data(location, weather_api_key)

    if "error" in weather_data:
        return weather_data["error"]

    # Prepare weather data for outfit suggestion
    weather_summary = f"Location: {weather_data['location']}, Temp: {weather_data['temperature']}°C, " \
                      f"Weather: {weather_data['weather']}, Humidity: {weather_data['humidity']}%, " \
                      f"Wind Speed: {weather_data['wind_speed']} m/s"

    # Generate outfit suggestion using LLAMA model
    outfit_suggestion = generate_outfit_suggestion(weather_summary)

    return outfit_suggestion

# Gradio Interface
interface = gr.Interface(
    fn=suggest_outfit,
    inputs=[
        gr.Textbox(label="Enter Your Location (e.g., London, New York)", placeholder="Location", lines=1),
        gr.Textbox(label="OpenWeatherMap API Key", type="password", placeholder="Weather API Key (Optional)")
    ],
    outputs="text",
    title="Real-Time Weather-Based Outfit Suggestion",
    description="Enter your location and get outfit suggestions based on current weather conditions."
)

interface.launch()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.6 MB/s eta 0:00:00
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c005aff62b76cf6b53.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


___

In [9]:
pip install streamlit openai requests


In [26]:
import requests
import streamlit as st
import groq

# Function to get weather data from OpenWeatherMap
def get_weather_data(city):
    api_key = "-----------------------------"  # Replace with your OpenWeatherMap API key
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    return response.json()

# Function to parse weather data
def parse_weather_data(weather_data):
    temperature = weather_data["main"]["temp"]
    weather_description = weather_data["weather"][0]["description"]
    return temperature, weather_description

# Function to get outfit suggestion using Groq's LLaMA model
def get_outfit_suggestion(temperature, description):
    model = groq.load_model("llama")  # Load LLaMA model from Groq
    prompt = f"The current weather is {description} with a temperature of {temperature}°C. Suggest an outfit."

    response = model.generate(
        prompt=prompt,
        max_length=100,
        temperature=0.7,  # Controls creativity of the response
    )

    return response['choices'][0]['text'].strip()

# Streamlit UI for user input
st.title("Weather-Based Outfit Suggestion App")

city = st.text_input("Enter your location:")

if city:
    weather_data = get_weather_data(city)

    if weather_data["cod"] == 200:
        temperature, description = parse_weather_data(weather_data)

        # Display current weather info
        st.write(f"Current temperature in {city}: {temperature}°C")
        st.write(f"Weather: {description}")

        # Get outfit suggestion from Groq's LLaMA model
        outfit_suggestion = get_outfit_suggestion(temperature, description)

        # Display outfit suggestion
        st.write("Outfit Suggestion:")
        st.write(outfit_suggestion)

        # Display weather icon
        icon_code = weather_data["weather"][0]["icon"]
        icon_url = f"http://openweathermap.org/img/wn/{icon_code}.png"
        st.image(icon_url)
    else:
        st.write("Could not retrieve weather data. Please check the location.")

# Optional: Add CSS for styling
st.markdown(
    """
    <style>
    .reportview-container {
        background: #f5f5f5;
    }
    .stButton>button {
        background-color: #ff5733;
        color: white;
    }
    </style>
    """,
    unsafe_allow_html=True
)


2024-11-09 14:41:25.534 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-09 14:41:25.604 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-11-09 14:41:25.607 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-09 14:41:25.610 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-09 14:41:25.613 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-09 14:41:25.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-09 14:41:25.617 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-09 14:41:25.618 Session state does not 

DeltaGenerator()

In [16]:
!wget -q -O - ipv4.icanhazip.com

34.83.23.169


In [18]:
!streamlit run app.py & npx localtunnel --port 8501

Streaming output truncated to the last 5000 lines.

diffusion_pytorch_model.safetensors:   0% 41.9M/10.3G [00:02<07:15, 23.5MB/s]
model.safetensors:   5% 126M/2.78G [00:02<01:17, 34.3MB/s]

model.safetensors:  13% 62.9M/492M [00:02<00:18, 22.7MB/s]




diffusion_pytorch_model.safetensors:  13% 41.9M/335M [00:02<00:12, 23.7MB/s]



diffusion_pytorch_model.safetensors:  13% 41.9M/335M [00:02<00:12, 23.3MB/s]


diffusion_pytorch_model.safetensors:   1% 52.4M/10.3G [00:03<12:52, 13.2MB/s]




diffusion_pytorch_model.safetensors:  16% 52.4M/335M [00:03<00:20, 13.8MB/s]
model.safetensors:   5% 136M/2.78G [00:03<02:15, 19.4MB/s]



diffusion_pytorch_model.safetensors:  16% 52.4M/335M [00:03<00:20, 13.8MB/s]

model.safetensors:  15% 73.4M/492M [00:03<00:30, 13.7MB/s]


diffusion_pytorch_model.safetensors:   1% 62.9M/10.3G [00:03<09:28, 18.0MB/s]




diffusion_pytorch_model.safetensors:  19% 62.9M/335M [00:03<00:14, 18.8MB/s]
model.safetensors:   5% 147M/2.78G [00:04<01:53, 23.2MB/s]

model.saf

In [ ]:
# import requests
# import streamlit as st
# import os
# from groq import Groq

# # Set the Groq API Key
# GROQ_API_KEY = "-----------------------"  # Replace with your Groq API key

# # Function to get weather data from OpenWeatherMap
# def get_weather_data(city):
#     api_key = "--------------------"  # Replace with your OpenWeatherMap API key
#     url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
#     response = requests.get(url)
#     return response.json()

# # Function to parse weather data
# def parse_weather_data(weather_data):
#     temperature = weather_data["main"]["temp"]
#     weather_description = weather_data["weather"][0]["description"]
#     return temperature, weather_description

# # Function to get outfit suggestion using Groq's LLaMA model
# def get_outfit_suggestion(temperature, description):
#     # Initialize Groq's API
#     client = Groq(api_key=GROQ_API_KEY)  # Pass the API key

#     prompt = f"The current weather is {description} with a temperature of {temperature}°C. Suggest an outfit."

#     # Use Groq's chat API for text generation
#     chat_completion = client.chat.completions.create(
#         messages=[{"role": "user", "content": prompt}],
#         model="llama3-8b-8192",  # Replace with the correct model if needed
#     )

#     return chat_completion.choices[0].message.content.strip()

# # Streamlit UI for user input
# st.title("Weather-Based Outfit Suggestion App")

# city = st.text_input("Enter your location:")

# if city:
#     weather_data = get_weather_data(city)

#     if weather_data["cod"] == 200:
#         temperature, description = parse_weather_data(weather_data)

#         # Display current weather info
#         st.write(f"Current temperature in {city}: {temperature}°C")
#         st.write(f"Weather: {description}")

#         # Get outfit suggestion from Groq's LLaMA model
#         outfit_suggestion = get_outfit_suggestion(temperature, description)

#         # Display outfit suggestion
#         st.write("Outfit Suggestion:")
#         st.write(outfit_suggestion)

#         # Display weather icon
#         icon_code = weather_data["weather"][0]["icon"]
#         icon_url = f"http://openweathermap.org/img/wn/{icon_code}.png"
#         st.image(icon_url)
#     else:
#         st.write("Could not retrieve weather data. Please check the location.")

# # Optional: Add CSS for styling
# st.markdown(
#     """
#     <style>
#     .reportview-container {
#         background: #f5f5f5;
#     }
#     .stButton>button {
#         background-color: #ff5733;
#         color: white;
#     }
#     </style>
#     """,
#     unsafe_allow_html=True
# )






import requests
import streamlit as st
import groq

# Function to get weather data from OpenWeatherMap
def get_weather_data(city):
    api_key = "---------------------------"  # Replace with your OpenWeatherMap API key
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    return response.json()

# Function to parse weather data
def parse_weather_data(weather_data):
    temperature = weather_data["main"]["temp"]
    weather_description = weather_data["weather"][0]["description"]
    return temperature, weather_description

# Function to get outfit suggestion using Groq's LLaMA model
def get_outfit_suggestion(temperature, description, style, fabric):
    # Initialize Groq's API
    client = groq.Groq(api_key="------------------------------------------")  # Replace with your Groq API key

    prompt = f"The current weather is {description} with a temperature of {temperature}°C. Suggest an outfit. The user prefers a {style} style and {fabric} fabric."

    # Use Groq's chat completion to get the text response
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",  # You can change to a different model if necessary
    )

    return response.choices[0].message.content.strip()

# Streamlit UI for user input
st.title("Weather-Based Outfit Suggestion App")

city = st.text_input("Enter your location:")

# Add style and fabric input options
style = st.selectbox("Select your preferred style", ["Casual", "Formal", "Sporty", "Business", "Chic"])
fabric = st.selectbox("Select your preferred fabric", ["Cotton", "Linen", "Wool", "Polyester", "Silk", "Leather"])

if city:
    weather_data = get_weather_data(city)

    if weather_data["cod"] == 200:
        temperature, description = parse_weather_data(weather_data)

        # Display current weather info
        st.write(f"Current temperature in {city}: {temperature}°C")
        st.write(f"Weather: {description}")

        # Get outfit suggestion based on user preferences
        outfit_suggestion = get_outfit_suggestion(temperature, description, style, fabric)

        # Display outfit suggestion
        st.write("Outfit Suggestion:")
        st.write(outfit_suggestion)

        # Display weather icon
        icon_code = weather_data["weather"][0]["icon"]
        icon_url = f"http://openweathermap.org/img/wn/{icon_code}.png"
        st.image(icon_url)
    else:
        st.write("Could not retrieve weather data. Please check the location.")

# Optional: Add CSS for styling
st.markdown(
    """
    <style>
    .reportview-container {
        background: #f5f5f5;
    }
    .stButton>button {
        background-color: #ff5733;
        color: white;
    }
    </style>
    """,
    unsafe_allow_html=True
)


In [10]:
!pip install diffusers transformers torch


In [11]:
import torch
print(torch.cuda.is_available())


True


In [13]:
from diffusers import StableDiffusionPipeline
import torch

# Load the pre-trained Stable Diffusion model
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")

# Check if GPU is available and use it, otherwise fall back to CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe.to(device)

# Example of generating an image
prompt = "A beautiful sunset over the mountains"
image = pipe(prompt).images[0]

# Show the image (optional in a Jupyter notebook)
image.show()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [32]:
# import requests
# import streamlit as st
# from diffusers import DiffusionPipeline
# import torch
# import groq

# # Function to get weather data from OpenWeatherMap
# def get_weather_data(city):
#     api_key = "-------------------------------------"  # Replace with your OpenWeatherMap API key
#     url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
#     response = requests.get(url)
#     return response.json()

# # Function to parse weather data
# def parse_weather_data(weather_data):
#     temperature = weather_data["main"]["temp"]
#     weather_description = weather_data["weather"][0]["description"]
#     return temperature, weather_description

# # Function to get outfit suggestion using Groq's LLaMA model
# def get_outfit_suggestion(temperature, description, style, fabric, gender):
#     # Initialize Groq's API
#     client = groq.Groq(api_key="-------------------------------------------")  # Replace with your Groq API key

#     prompt = f"The current weather is {description} with a temperature of {temperature}°C. Suggest an outfit for a {gender} with {style} style and {fabric} fabric."

#     # Use Groq's chat completion to get the text response
#     response = client.chat.completions.create(
#         messages=[{
#             "role": "user",
#             "content": prompt,
#         }],
#         model="llama3-8b-8192",  # You can change to a different model if necessary
#     )

#     return response.choices[0].message.content.strip()

# # Function to generate an outfit image based on the description
# def generate_outfit_image(outfit_description):
#     # Load the pre-trained model from Hugging Face
#     pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0")
#     pipe.load_lora_weights("Pinguin/luisap-not-dalle2-27mb")
#     pipe.to("cuda")  # Ensure the model runs on GPU, if available

#     # Generate the image from the description
#     image = pipe(outfit_description).images[0]

#     return image

# def get_shopping_links(outfit_suggestion):
#     # Extract key items from the outfit suggestion
#     keywords = []
#     if "cotton kameez" in outfit_suggestion.lower():
#         keywords.append("cotton kameez")
#     if "pajama pants" in outfit_suggestion.lower():
#         keywords.append("pajama pants")
#     if "shawl" in outfit_suggestion.lower():
#         keywords.append("cotton shawl")
#     if "loafers" in outfit_suggestion.lower():
#         keywords.append("loafers")
#     if "ballet flats" in outfit_suggestion.lower():
#         keywords.append("ballet flats")
#     if "sandals" in outfit_suggestion.lower():
#         keywords.append("sandals")

#     # Generate search query based on extracted keywords
#     search_query = '+'.join(keywords)
#     shopping_url = f"https://www.amazon.com/s?k={search_query.replace(' ', '+')}"

#     return shopping_url


# # Streamlit UI for user input
# st.title("Weather-Based Outfit Suggestion App")

# # Multilingual Support: Translate interface (Example: English & Spanish)
# language = st.selectbox("Select Language", ["English", "Spanish"])

# if language == "Spanish":
#     st.header("Sugerencia de Atuendo Basada en el Clima")
#     city_label = "Ingrese su ubicación:"
#     style_label = "Seleccione su estilo preferido"
#     fabric_label = "Seleccione su tela preferida"
#     gender_label = "Seleccione su género"
#     shopping_label = "Encuentra atuendos aquí"
# else:
#     st.header("Weather-Based Outfit Suggestion")
#     city_label = "Enter your location:"
#     style_label = "Select your preferred style"
#     fabric_label = "Select your preferred fabric"
#     gender_label = "Select your gender"
#     shopping_label = "Find outfits here"

# # Sidebar: Left panel for customer selection
# with st.sidebar:
#     st.subheader("Customer Selection")

#     city = st.text_input(city_label, placeholder="Type your city...")

#     # Gender selection dropdown
#     gender_options = ["Male", "Female", "Other"]
#     gender = st.selectbox(gender_label, gender_options)

#     # Style and fabric options
#     style_options = ["Casual", "Formal", "Sporty", "Business", "Chic", "Trendy", "Streetwear", "Bohemian"]
#     fabric_options = ["Cotton", "Linen", "Wool", "Polyester", "Silk", "Leather", "Denim", "Rayon"]

#     style = st.selectbox(style_label, style_options)
#     fabric = st.selectbox(fabric_label, fabric_options)

#     # Custom style and fabric input
#     custom_style = st.text_input("Or enter your own style (optional):", placeholder="e.g. Vintage")
#     custom_fabric = st.text_input("Or enter your own fabric (optional):", placeholder="e.g. Cashmere")

#     if custom_style:
#         style = custom_style
#     if custom_fabric:
#         fabric = custom_fabric

# # Main content area for displaying weather and outfit suggestions
# if city:
#     weather_data = get_weather_data(city)

#     if weather_data["cod"] == 200:
#         temperature, description = parse_weather_data(weather_data)

#         # Display current weather info with chatbot-like bubbles
#         st.markdown(f"<div class='chat-bubble'>Current temperature in {city}: {temperature}°C</div>", unsafe_allow_html=True)
#         st.markdown(f"<div class='chat-bubble'>{description.capitalize()}</div>", unsafe_allow_html=True)

#         # Get outfit suggestion based on user preferences, including gender
#         outfit_suggestion = get_outfit_suggestion(temperature, description, style, fabric, gender)

#         # Display outfit suggestion in a chat bubble
#         st.markdown(f"<div class='chat-bubble'><strong>Suggested Outfit:</strong> {outfit_suggestion}</div>", unsafe_allow_html=True)

#         # Generate and display the outfit image
#         outfit_image = generate_outfit_image(outfit_suggestion)
#         st.image(outfit_image, caption="Generated Outfit Image", use_column_width=True)

#         # Get shopping link for the suggested outfit
#         shopping_link = get_shopping_links(outfit_suggestion)
#         st.markdown(f"[Buy this outfit: {shopping_link}]({shopping_link})", unsafe_allow_html=True)

#         # Display weather icon
#         icon_code = weather_data["weather"][0]["icon"]
#         icon_url = f"http://openweathermap.org/img/wn/{icon_code}.png"
#         st.image(icon_url, width=50)

#         # Weather alerts
#         if temperature > 30:
#             st.warning(f"Warning: It's quite hot in {city}, stay hydrated!")
#         elif temperature < 10:
#             st.warning(f"Warning: It's cold in {city}, dress warmly!")
#     else:
#         st.write("Could not retrieve weather data. Please check the location.")

# # Optional: Add CSS for styling
# st.markdown(
#     """
#     <style>
#     .reportview-container {
#         background: #e0f7fa;
#         display: flex;
#         flex-direction: column;
#         justify-content: center;
#         align-items: center;
#     }

#     .stButton>button {
#         background-color: #ff5733;
#         color: white;
#         border-radius: 20px;
#         padding: 10px 20px;
#         font-size: 16px;
#         transition: background-color 0.3s ease;
#     }

#     .stButton>button:hover {
#         background-color: #e34b1f;
#     }

#     .stSelectbox {
#         background-color: #f1f1f1;
#         border-radius: 5px;
#         width: 250px;
#         font-size: 14px;
#     }

#     .chat-bubble {
#         background-color: #ffffff;
#         padding: 10px;
#         border-radius: 10px;
#         max-width: 70%;
#         margin: 5px 0;
#         box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
#         font-size: 16px;
#         font-family: 'Roboto', sans-serif;
#     }

#     h1, h2 {
#         font-family: 'Roboto', sans-serif;
#         color: #333333;
#     }

#     .stTextInput input {
#         font-size: 14px;
#     }
#     </style>
#     """, unsafe_allow_html=True)


2024-11-09 16:26:12.111 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-09 16:26:12.115 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-09 16:26:12.118 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-09 16:26:12.120 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-09 16:26:12.122 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-09 16:26:12.125 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-09 16:26:12.126 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-09 16:26:12.128 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()